In [1]:
!pip install Biopython

You should consider upgrading via the 'c:\users\minie\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import tarfile
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Dense, Bidirectional, TimeDistributed

from matplotlib import pyplot as plt

from Bio.Seq import Seq

### Read data

In [3]:
filename = "training_data.tar.gz"

data = tarfile.open(filename, "r:gz")
data.extractall()
data.close()

In [4]:
b = open('ghl_gold.fa','r')
bind = b.readlines()
b.close()

u = open('ghl_gold_random.fa','r')
unbind = u.readlines()
u.close()

### Data preprocessing

In [5]:
bind = [v for v in bind if '>' not in v]
bind = [s.replace('\n', '') for s in bind]
bind = [x.upper() for x in bind]

unbind = [v for v in unbind if '>' not in v]
unbind = [s.replace('\n', '') for s in unbind]
unbind = [x.upper() for x in unbind]

In [6]:
print(len(bind), len(unbind))

1400090 1400090


##### Reverse complement

In [7]:
bind_rev = list(range(len(bind)))

for i in range(len(bind)):
  seq = Seq(bind[i])
  rev = seq.reverse_complement()
  bind_rev[i] = str(rev)

unbind_rev = list(range(len(unbind)))

for i in range(len(unbind)):
  seq = Seq(unbind[i])
  rev = seq.reverse_complement()
  unbind_rev[i] = str(rev)

In [8]:
bind_fb = bind + bind_rev
unbind_fb = unbind + unbind_rev

In [9]:
bind_label = [1 for i in range(len(bind_fb))]
unbind_label = [0 for i in range(len(unbind_fb))]

In [10]:
bind_dict = {"seq":bind_fb, "label":bind_label}
unbind_dict = {"seq":unbind_fb, "label":unbind_label}

In [11]:
bind_df = pd.DataFrame(bind_dict)
unbind_df = pd.DataFrame(unbind_dict)

In [12]:
df = pd.concat([bind_df, unbind_df])

##### split the dataset

In [13]:
from sklearn.utils import shuffle

new_df = shuffle(df)
new_df = new_df.reset_index()

In [14]:
x = new_df.seq
y = new_df.label

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40)

##### One-hot Encoding

In [16]:
LE = LabelEncoder()
LE.fit(['A', 'C', 'G', 'T', 'N'])

LabelEncoder()

In [17]:
start = datetime.now()

for index, row in x_train.items():
  x_train[index] = LE.transform(list(row))

for index, row in x_test.items():
  x_test[index] = LE.transform(list(row))

x_train = to_categorical(x_train.values.tolist())
x_t = to_categorical(x_test.values.tolist())

y_train = to_categorical(y_train.values.tolist())
y_t = to_categorical(y_test.values.tolist())

end = datetime.now()
print("encoding running time : "+str(end-start))

encoding running time : 0:07:43.316557


### CNN

In [18]:
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=7, strides=1, padding='valid', input_shape=(20,5), activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3, strides=1, padding='valid'))
cnn_model.add(Flatten())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(2, activation='sigmoid'))
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
cnn_history = cnn_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

Epoch 1/10
112008/112008 [==============================] - 384s 3ms/step - loss: 0.3384 - accuracy: 0.8542 - val_loss: 0.3278 - val_accuracy: 0.8584
Epoch 2/10
112008/112008 [==============================] - 383s 3ms/step - loss: 0.3350 - accuracy: 0.8557 - val_loss: 0.3268 - val_accuracy: 0.8585
Epoch 3/10
112008/112008 [==============================] - 400s 4ms/step - loss: 0.3337 - accuracy: 0.8564 - val_loss: 0.3253 - val_accuracy: 0.8595
Epoch 4/10
112008/112008 [==============================] - 412s 4ms/step - loss: 0.3331 - accuracy: 0.8566 - val_loss: 0.3257 - val_accuracy: 0.8591
Epoch 5/10
112008/112008 [==============================] - 413s 4ms/step - loss: 0.3325 - accuracy: 0.8569 - val_loss: 0.3239 - val_accuracy: 0.8598
Epoch 6/10
112008/112008 [==============================] - 420s 4ms/step - loss: 0.3322 - accuracy: 0.8570 - val_loss: 0.3268 - val_accuracy: 0.8582
Epoch 7/10
112008/112008 [==============================] - 415s 4ms/step - loss: 0.3319 - accuracy:

### RNN

LSTM

In [21]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(20, 5), return_sequences=True))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(2, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
lstm_history = lstm_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

Epoch 1/10
 11171/112008 [=>............................] - ETA: 1:26:57 - loss: 0.4179 - accuracy: 0.8058- ETA: 1:26:56 - loss: 0.4180 - accur

KeyboardInterrupt: 

Bi-LSTM

In [ ]:
bi_model = Sequential()
bi_model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(20, 5)))
bi_model.add(Bidirectional(LSTM(64)))
bi_model.add(Dense(64, activation='relu'))
bi_model.add(Dropout(0.2))
bi_model.add(Dense(2, activation='sigmoid'))
bi_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bi_history = bi_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

### CNN + RNN

CNN + LSTM

In [ ]:
cl_model = Sequential()
cl_model.add(Conv1D(filters=64, kernel_size=7, strides=1, padding='valid', activation='relu', input_shape=(20, 5)))
cl_model.add(MaxPooling1D(pool_size=3, strides=1, padding='valid'))

cl_model.add(LSTM(128, return_sequences=True))
cl_model.add(LSTM(128))
cl_model.add(Dense(64, activation='relu'))
cl_model.add(Dropout(0.2))
cl_model.add(Dense(2, activation='sigmoid'))
cl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cl_history = cl_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

CNN + Bi-LSTM

In [ ]:
cbi_model = Sequential()
cbi_model.add(Conv1D(filters=64, kernel_size=7, strides=1, padding='valid', activation='relu', input_shape=(20, 5)))
cbi_model.add(MaxPooling1D(pool_size=3, strides=1, padding='valid'))

cbi_model.add(Bidirectional(LSTM(64, return_sequences=True)))
cbi_model.add(Bidirectional(LSTM(64)))
cbi_model.add(Dense(64, activation='relu'))
cbi_model.add(Dropout(0.2))
cbi_model.add(Dense(2, activation='sigmoid'))
cbi_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cbi_history = cbi_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

### Evaluation

Accuracy

In [ ]:
cnn_score = cnn_model.evaluate(x_t, y_t)
lstm_score = lstm_model.evaluate(x_t, y_t)
bi_score = bi_model.evaluate(x_t, y_t)
cl_score = cl_model.evaluate(x_t, y_t)
cbi_score = cbi_model.evaluate(x_t, y_t)

model = ['CNN', 'LSTM', "BiLSTM", "CNN-LSTM", "CNN-BiLSTM"]
acc = [round(cnn_score[1], 2), round(lstm_score[1], 2), round(bi_score[1], 2), round(ci_score[1], 2), round(cbi_score[1], 2)]
acc_dict = {"Model":model, "Accuracy":bind_label}
df_acc = pd.DataFrame(acc_dict)

df_acc

loss-epoch curve

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(25,5))

axs[0].plot(cnn_history.history['loss'])
axs[0].plot(cnn_history.history['val_loss'])
axs[0].set(ylabel='Loss')
axs[0].set_title('CNN')

axs[1].plot(lstm_history.history['loss'])
axs[1].plot(lstm_history.history['val_loss'])
axs[1].set_title('LSTM')

axs[2].plot(bi_history.history['loss'])
axs[2].plot(bi_history.history['val_loss'])
axs[2].set_title('BiLSTM')

axs[3].plot(cl_history.history['loss'])
axs[3].plot(cl_history.history['val_loss'])
axs[3].set_title('CNN-LSTM')

axs[4].plot(cbi_history.history['loss'])
axs[4].plot(cbi_history.history['val_loss'])
axs[4].set_title('CNN-BiLSTM')

for ax in axs.flat:
    ax.set(xlabel='Epoch')

plt.suptitle('Model Loss')
plt.legend(['train', 'val'], loc='upper right', bbox_to_anchor=(1.05, 1.0))

precision-recall curve

In [ ]:
cnn_probs = cnn_model.predict(x_t)[:,1]
lstm_probs = lstm_model.predict(x_t)[:,1]
bi_probs = bi_model.predict(x_t)[:,1]
cl_probs = cl_model.predict(x_t)[:,1]
cbi_probs = cbi_model.predict(x_t)[:,1]

In [ ]:
cnn_precision, cnn_recall, cnn_thresholds = precision_recall_curve(y_test.values, cnn_probs)
lstm_precision, lstm_recall, lstm_thresholds = precision_recall_curve(y_test.values, lstm_probs)
bi_precision, bi_recall, bi_thresholds = precision_recall_curve(y_test.values, bi_probs)
cl_precision, cl_recall, cl_thresholds = precision_recall_curve(y_test.values, cl_probs)
cbi_precision, cbi_recall, cbi_thresholds = precision_recall_curve(y_test.values, cbi_probs)

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(25,3))

axs[0].plot(cnn_precision, cnn_recall)
#axs[0].set(ylabel='Loss')
axs[0].set_title('CNN')

axs[1].plot(lstm_precision, lstm_recall)
axs[1].set_title('LSTM')

axs[2].plot(bi_precision, bi_recall)
axs[2].set_title('BiLSTM')

axs[3].plot(cl_precision, cl_recall)
axs[3].set_title('CNN-LSTM')

axs[4].plot(cbi_precision, cbi_recall)
axs[4].set_title('CNN-BiLSTM')

#for ax in axs.flat:
#    ax.set(xlabel='Epoch')

plt.suptitle('Precision-Recall Curve')
# plt.legend(['train', 'val'], loc='upper right', bbox_to_anchor=(1.05, 1.0))

ROC Curve, AUC

In [ ]:
cnn_auc = roc_auc_score(y_test.values, cnn_probs)
cnn_fpr, cnn_tpr, cnn_ = roc_curve(y_test.values, cnn_probs)

lstm_auc = roc_auc_score(y_test.values, lstm_probs)
lstm_fpr, lstm_tpr, lstm_ = roc_curve(y_test.values, lstm_probs)

bi_auc = roc_auc_score(y_test.values, bi_probs)
bi_fpr, bi_tpr, bi_ = roc_curve(y_test.values, bi_probs)

cl_auc = roc_auc_score(y_test.values, cl_probs)
cl_fpr, cl_tpr, cl_ = roc_curve(y_test.values, cl_probs)

cbi_auc = roc_auc_score(y_test.values, cbi_probs)
cbi_fpr, cbi_tpr, cbi_ = roc_curve(y_test.values, cbi_probs)

In [ ]:
plt.plot(cnn_fpr, cnn_tpr, color='salmon', linewidth=2)
plt.plot(lstm_fpr, lstm_tpr, color='coral', linewidth=2)
plt.plot(bi_fpr, bi_tpr, color='gold', linewidth=2)
plt.plot(cl_fpr, cl_tpr, color='forestgreen', linewidth=2)
plt.plot(cbi_fpr, cbi_tpr, color='cornflowerblue', linewidth=2)

# lightcoral salmon coral darkorange crimson deepskyblue gold forestgreen seagreen cornflowerblue darkorchid

plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(['CNN (AUC = ' + str(round(cnn_auc,4)) + ')', 'LSTM (AUC = ' + str(round(lstm_auc,4)) + ')',
            'BiLSTM (AUC = ' + str(round(bi_auc,4)) + ')', 'CNN-LSTM (AUC = ' + str(round(cl_auc,4)) + ')',
            'CNN-BiLSTM (AUC = ' + str(round(cbi_auc,4)) + ')'], loc='lower right')

plt.show()